In [17]:
from sklearn import tree #decision tree
from sklearn.ensemble import RandomForestRegressor #random Forest
from sklearn.model_selection import cross_val_score # evaluate our model
from sklearn.model_selection import GridSearchCV # gridsearch to find best hyper parameters
import pandas as pd
import numpy as np
import datetime as dt

In [18]:
Schedule_and_stats_played = pd.read_csv("2020_2021_Played_Games.csv", index_col=0)
Schedule_and_stats_unplayed = pd.read_csv("2020_2021_Unplayed_Games.csv", index_col=0)

In [19]:
Schedule_and_stats_played["Date"] = pd.to_datetime(Schedule_and_stats_played["Date"])
Schedule_and_stats_unplayed["Date"] = pd.to_datetime(Schedule_and_stats_unplayed["Date"])


In [20]:
Schedule_and_stats_played[(Schedule_and_stats_played['home_neutral']=='Michigan')]

,Date,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
893,2020-11-25,Bowling Green State,82,Michigan,96,Bowling Green State,19,10,9,0.526,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
894,2020-12-02,Ball State,65,Michigan,84,Ball State,16,7,9,0.438,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
895,2020-11-29,Oakland,71,Michigan,81,Oakland,23,8,15,0.348,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
896,2020-12-09,Toledo,71,Michigan,91,Toledo,21,16,5,0.762,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
897,2021-01-12,Wisconsin,54,Michigan,77,Wisconsin,21,15,6,0.714,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
898,2020-12-06,UCF,58,Michigan,80,UCF,15,5,10,0.333,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
899,2020-12-13,Penn State,58,Michigan,62,Penn State,16,7,9,0.438,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
900,2021-01-19,Maryland,63,Michigan,87,Maryland,20,10,10,0.500,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
901,2021-02-18,Rutgers,64,Michigan,71,Rutgers,18,11,7,0.611,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
902,2021-01-06,Minnesota,57,Michigan,82,Minnesota,20,13,7,0.650,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0


In [5]:
Schedule_and_stats_unplayed[(Schedule_and_stats_unplayed['visitor_neutral']=='Michigan') | (Schedule_and_stats_unplayed['home_neutral']=='Michigan')]

,Date,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
60,2021-03-02,Illinois,0,Michigan,0,Illinois,18,13,5,0.722,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
61,2021-02-25,Iowa,0,Michigan,0,Iowa,20,14,6,0.700,...,179.0,233.0,0.768,128.0,555.0,243.0,65.0,73.0,181.0,212.0
251,2021-02-27,Michigan,0,Indiana,0,Michigan,14,13,1,0.929,...,301.0,453.0,0.664,167.0,682.0,275.0,111.0,84.0,226.0,341.0


In [6]:
X = Schedule_and_stats_played.drop(["Date","visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1)
Y = Schedule_and_stats_played[["PTS","PTS.1"]]

In [7]:
reg = tree.DecisionTreeRegressor(random_state=0)
reg = reg.fit(X, Y)

In [8]:
### cross validations
print(cross_val_score(reg, X, Y, cv=5))

#First model performs HORRIBLY!

[-0.58441457 -0.53766363 -0.53160381 -0.62721973 -0.64667568]


In [9]:
# use grid search to determine decision tree depth
params = {"max_depth":range(1,10)}
treereg = tree.DecisionTreeRegressor(random_state=0)
reg2 = GridSearchCV(estimator = treereg, param_grid=params)
reg2.fit(X,Y)
reg2.best_params_


{'max_depth': 3}

In [10]:
means = reg2.cv_results_['mean_test_score']
stds = reg2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()


0.027 (+/-0.029) for {'max_depth': 1}

0.079 (+/-0.046) for {'max_depth': 2}

0.091 (+/-0.048) for {'max_depth': 3}

0.088 (+/-0.062) for {'max_depth': 4}

0.081 (+/-0.062) for {'max_depth': 5}

0.046 (+/-0.062) for {'max_depth': 6}

-0.027 (+/-0.083) for {'max_depth': 7}

-0.132 (+/-0.052) for {'max_depth': 8}

-0.225 (+/-0.043) for {'max_depth': 9}



In [11]:
reg3 = RandomForestRegressor(random_state=0)
reg3.fit(X,Y)
print(cross_val_score(reg3, X, Y, cv=5))

[0.21022582 0.22416048 0.1737839  0.20076599 0.17137106]


In [12]:
#get game to predict.  Gonzaga vs Northwestern state
month, day, year = 2, 13, 2021
date = pd.to_datetime(str(year)+"-"+str(month)+"-"+str(day))
games = Schedule_and_stats_unplayed[Schedule_and_stats_unplayed["Date"]==date]
teams = games[["visitor_neutral", "home_neutral"]]
games = games.drop(["Date","visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1)
games

,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y


In [13]:
with pd.option_context('display.max_columns', None):
  display(Schedule_and_stats_unplayed.head(2))

,Date,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,Away W_x,Away L_x,Tm._x,Opp._x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,ORB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,School_y,G_y,W_y,L_y,W-L%_y,SRS_y,SOS_y,Conf W_y,Conf L_y,Home W_y,Home L_y,Away W_y,Away L_y,Tm._y,Opp._y,MP_y,FG_y,FGA_y,FG%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,2021-02-22,Texas Tech,0,Oklahoma State,0,Texas Tech,20,14,6,0.700,16.66,5.11,6,5,8,4,5,1,1485.0,1254.0,805.0,505.0,1158.0,0.436,128.0,377.0,0.340,347.0,479.0,0.724,244.0,715.0,254.0,144.0,84.0,220.0,349.0,Oklahoma State,18,12,6,0.667,12.17,7.84,5,6,6,3,6,3,1372.0,1294.0,735.0,490.0,1077.0,0.455,116.0,344.0,0.337,276.0,399.0,0.692,208.0,712.0,233.0,131.0,82.0,280.0,313.0
1,2021-03-01,Oklahoma,0,Oklahoma State,0,Oklahoma,17,12,5,0.706,16.63,8.10,7,4,10,1,2,4,1284.0,1139.0,680.0,459.0,1034.0,0.444,134.0,398.0,0.337,232.0,313.0,0.741,179.0,636.0,237.0,130.0,57.0,183.0,242.0,Oklahoma State,18,12,6,0.667,12.17,7.84,5,6,6,3,6,3,1372.0,1294.0,735.0,490.0,1077.0,0.455,116.0,344.0,0.337,276.0,399.0,0.692,208.0,712.0,233.0,131.0,82.0,280.0,313.0


In [14]:
predictions = reg3.predict(games)

ValueError: Found array with 0 sample(s) (shape=(0, 62)) while a minimum of 1 is required.

In [ ]:
predictions = pd.DataFrame(predictions, columns = ["score_visitor", "score_home"])

In [ ]:
predictions.head()

In [ ]:
teams = teams.reset_index(drop=True)
predictions = predictions.reset_index(drop=True)
predictions.head(5)

In [ ]:
predictions = teams.join(predictions, how='inner')

In [ ]:
predictions.columns

In [ ]:
predictions["line"] = predictions["score_home"] - predictions["score_visitor"]

In [ ]:
predictions